In [1]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import json
from tqdm import tqdm
import re

In [ ]:
username = 'elastic'
password = 'stimulus5affect-roof'
host = 'http://128.143.69.88:9200'
index_name = "cadets*"

# Initialize Elasticsearch client
es = Elasticsearch([host], http_auth=(username, password))

# Test connection and index existence
if not es.ping():
    print("Elasticsearch cluster is not accessible!")
else:
    print("Connected to Elasticsearch.")
if not es.indices.exists(index=index_name):
    print(f"Index {index_name} does not exist.")
else:
    print(f"Index {index_name} exists.")

# Define the query
query = {"query": {"match_all": {}}}

# Get total count of documents to process
total_docs = es.count(index=index_name, body=query)['count']
print(f"Total documents to process: {total_docs}")

edge_types = set([
    'EVENT_CLOSE',
    'EVENT_OPEN',
    'EVENT_READ',
    'EVENT_WRITE',
    'EVENT_EXECUTE',
    'EVENT_RECVFROM',
    'EVENT_RECVMSG',
    'EVENT_SENDMSG',
    'EVENT_SENDTO',
])

# Define file paths
id_to_type_file = 'e5_data/cadets_id_to_type.json'
net2prop_file = 'e5_data/cadets_net2prop.json'

# Define file paths
id_to_type_file = 'e5_data/cadets_id_to_type.json'
net2prop_file = 'e5_data/cadets_net2prop.json'
info_file = 'e5_data/cadets_info.json'

# Initialize buffers
net2prop_buffer = []
id_to_type_buffer = []
info_buffer = []
buffer_size = 1000000  # Change this number to adjust the buffer size

# Function to append data to a file
def append_to_file(file_path, data):
    with open(file_path, 'a') as file:
        for item in data:
            file.write(json.dumps(item) + '\n')

# Function to check buffer and flush if necessary
def check_and_flush_buffer():
    global net2prop_buffer, id_to_type_buffer, info_buffer
    if len(net2prop_buffer) >= buffer_size:
        append_to_file(net2prop_file, net2prop_buffer)
        net2prop_buffer = []
    if len(id_to_type_buffer) >= buffer_size:
        append_to_file(id_to_type_file, id_to_type_buffer)
        id_to_type_buffer = []
    if len(info_buffer) >= buffer_size:
        append_to_file(info_file, info_buffer)
        info_buffer = []
        
# Start processing documents
with tqdm(total=total_docs, desc="Processing Documents") as pbar:
    for doc in helpers.scan(es, query=query, index=index_name, size=1000):
        pbar.update(1)
        
        line = doc['_source']
        str_line = json.dumps(line)
        
        if "avro.cdm20.NetFlowObject" in str_line:
            net_flow_object = line['datum']['com.bbn.tc.schema.avro.cdm20.NetFlowObject']
            try:
                nodeid = net_flow_object['uuid']
                srcaddr = net_flow_object['localAddress'].get('string')
                srcport = net_flow_object['localPort'].get('int')
                dstaddr = net_flow_object['remoteAddress'].get('string')
                dstport = net_flow_object['remotePort'].get('int')

                net2prop_data = {nodeid: [srcaddr, srcport, dstaddr, dstport]}
                id_to_type_data = {nodeid: 'NETFLOW'}
                net2prop_buffer.append(net2prop_data)
                id_to_type_buffer.append(id_to_type_data)
            except: 
                pass

        if "schema.avro.cdm20.Subject" in str_line:
            subject = line['datum']['com.bbn.tc.schema.avro.cdm20.Subject']
            uuid = subject['uuid']
            record_type = subject['type'] 
            id_to_type_data = {uuid: record_type}
            id_to_type_buffer.append(id_to_type_data)

        if "schema.avro.cdm20.FileObject" in str_line:
            file_object = line['datum']['com.bbn.tc.schema.avro.cdm20.FileObject']
            uuid = file_object['uuid']
            file_type = file_object['type']
            id_to_type_data = {uuid: file_type}
            id_to_type_buffer.append(id_to_type_data)
            
        try:
            x = line
            
            try:
                action = x['datum']['com.bbn.tc.schema.avro.cdm20.Event']['type']
            except KeyError:
                action = ''
            try:
                actor = x['datum']['com.bbn.tc.schema.avro.cdm20.Event']['subject']['com.bbn.tc.schema.avro.cdm20.UUID']
            except KeyError:
                actor = ''
            try:
                obj = x['datum']['com.bbn.tc.schema.avro.cdm20.Event']['predicateObject']['com.bbn.tc.schema.avro.cdm20.UUID']
            except KeyError:
                obj = ''
            try:
                cmd = x['datum']['com.bbn.tc.schema.avro.cdm20.Event']['properties']['map']['exec']
            except KeyError:
                cmd = ''
            try:
                path = x['datum']['com.bbn.tc.schema.avro.cdm20.Event']['predicateObjectPath']['string']
            except KeyError:
                path = ''
            try:
                timestampnano = x['datum']['com.bbn.tc.schema.avro.cdm20.Event']['timestampNanos']
                timestamp = x['@timestamp']
            except KeyError:
                timestamp = ''
                timestampnano = ''

            if action in edge_types:
                info_data = {'actorID': actor, 'objectID': obj, 'action': action, 'timestampNanos': timestampnano, 'timestamp': timestamp, 'exec': cmd, 'path': path, 'hostid': x['hostId']}
                info_buffer.append(info_data)
        except:
            pass
        
        check_and_flush_buffer()
        
append_to_file(net2prop_file, net2prop_buffer)
append_to_file(id_to_type_file, id_to_type_buffer)
append_to_file(info_file, info_buffer)

/tmp/ipykernel_264608/3095134833.py:7: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch([host], http_auth=(username, password))


Connected to Elasticsearch.
Index cadets* exists.
Total documents to process: 1217886929


Processing Documents:   0%|          | 3247001/1217886929 [03:44<22:42:25, 14858.77it/s]